In [1]:
import os
import sys
import re

import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sys.path.insert(0, '../src')

from utils import *
from features import *

In [4]:
sns.set(rc={'figure.figsize':(20,5)})

In [5]:
mbit_rate = 1/125000

low_fp = '../data/240p/' 
med_fp = '../data/480p/'
high_fp = '../data/1080p/' 

In [6]:
low_dfs = []
for file in os.listdir(low_fp):
    if file != '.ipynb_checkpoints':
        low_dfs.append(pd.read_csv(low_fp+file))
    
med_dfs = []
for file in os.listdir(med_fp):
    if file != '.ipynb_checkpoints':
        med_dfs.append(pd.read_csv(med_fp+file))
    
high_dfs = []
for file in os.listdir(high_fp):
    if file != '.ipynb_checkpoints':
        high_dfs.append(pd.read_csv(high_fp+file))

## Aggregate Features

In [ ]:
## take the aggregate features of the whole chunk; download and upload
def agg_feat(df, col):
  return [np.mean(df[col]), np.std(df[col])]

## take the ratio of upload:download packets
def pkt_ratio(df):
  ms_df = convert_ms_df(df, True)
  local = np.sum(ms_df['pkt_src'] == '1') 
  server = np.sum(ms_df['pkt_src'] == '2') 
  return local / server

## take the ratio of upload:download bytes
def bytes_ratio(df):
  local = df['1->2Bytes'].sum()
  server = df['2->1Bytes'].sum()
  return local / server

## Peak Related Aggregate Features

In [ ]:
## finds the peaks with mean + 2(1) std
## run the above aggregate functions on the peaks only??

def get_peak_loc(df, col, invert=False):
  'invert arg allows you to get values not considered peaks'
  df_avg = df[col].mean()
  df_std = df[col].std()
  
  threshold = df_avg + (1 * df_std)
  
  if invert:
    return np.array(df[col] < threshold)
  
  else:
    return np.array(df[col] > threshold)

## np.mean, np.var, np.std - think of more?  
def peak_time_diff(df, col):
  '''
  mess around with the different inputs for function. 
  variance seems to inflate the difference betweent the two the most with litte
  to no data manipulation. however, currently trying things like
  squaring the data before taking the aggregate function to exaggerate
  differences (moderate success??)
  '''
  peaks = df[get_peak_loc(df, col)]
  peaks['Time'] = peaks['Time'] - peaks['Time'].min()
  time_diff = np.diff(peaks['Time'] ** 2)
  return [np.mean(time_diff), np.var(time_diff), np.std(time_diff)]

In [ ]:
from scipy.signal import find_peaks

def peak_times(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    if list(peaks) == []:
        return [-1]
    times = df.iloc[peaks]['Time'].values
    time_between_peaks = [times[i]-times[i-1]for i in range(1,len(times))]
    #print(time_between_peaks)
    #time_between_peaks[0]=0
    if time_between_peaks == []:
        return -1
    return time_between_peaks

def num_peaks(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    return len(peaks)

## Spectral Features

In [ ]:
def spectral_features(df, col):

    """
    welch implemention of spectral features
    resample the data before inputting (might change prereq depending on
    resource allocation)
    """

    f, Pxx_den = sp.signal.welch(df[col], fs=2)
    Pxx_den = np.sqrt(Pxx_den)

    peaks = sp.signal.find_peaks(Pxx_den)[0]
    prominences = sp.signal.peak_prominences(Pxx_den, peaks)[0]

    idx_max = prominences.argmax()
    loc_max = peaks[idx_max]

    return [f[loc_max], Pxx_den[loc_max], prominences[idx_max]]

## Chunking & Feature creation

In [ ]:
## wip; need to decide chunk size eventually
## should we also make this chunking feature be our feature creation?

def chunk_data(df, interval=60):

    """
    takes in a filepath to the data you want to chunk and feature engineer
    chunks our data into a specified time interval
    each chunk is then turned into an observation to be fed into our classifier
    """

    df_list = []
    
    df['Time'] = df['Time'] - df['Time'].min()
    
    total_chunks = np.floor(df['Time'].max() / interval).astype(int)

    for chunk in np.arange(total_chunks):
      
        start = chunk * interval
        end = (chunk+1) * interval

        temp_df = (df[(df['Time'] >= start) & (df['Time'] < end)])
        
        df_list.append(temp_df)
        
    return df_list

In [9]:
def create_features(df, interval=60):

  features = [
    'dwl_peak_freq',
    'dwl_peak_prom',
    'dwl_max_psd',
    'dwl_bytes_avg',
    'dwl_bytes_std',
    'dwl_peak_avg',
    'dwl_peak_var',
    'dwl_peak_std',
    'upl_peak_freq',
    'upl_peak_prom',
    'upl_max_psd',
    'upl_bytes_avg',
    'upl_bytes_std',
    'upl_peak_avg',
    'upl_peak_var',
    'upl_peak_std',
    'IMAN_dwn_time_peak',#'IMAN_up_time_peak',
    'IMAN_dwn_num_peak'#,'IMAN_up_num_peak'
  ]  

  vals = []

  df_chunks = chunk_data(df, interval)

  for chunk in df_chunks:

    preproc = convert_ms_df(chunk, True)
    upl_bytes = preproc[preproc['pkt_src'] == '1'].resample('500ms', on='Time').sum()
    dwl_bytes = preproc[preproc['pkt_src'] == '2'].resample('500ms', on='Time').sum()

    ## spectral features
    dwl_spectral = spectral_features(dwl_bytes, 'pkt_size')
    upl_spectral = spectral_features(upl_bytes, 'pkt_size')
    
    ## aggregate features
    dwl_agg = agg_feat(chunk, '2->1Bytes')
    upl_agg = agg_feat(chunk, '1->2Bytes')
    
    ## peak features
    dwl_peak = peak_time_diff(chunk, '2->1Bytes')
    upl_peak = peak_time_diff(chunk, '1->2Bytes')
    
    ## iman's time between peak 
    iman_dwn_time_peak = np.mean(peak_times(chunk,'2->1Bytes',1000000))
    #iman_up_time_peak = np.mean(peak_times(chunk,'1->2Bytes',50000))

    ## iman's num peak
    iman_dwn_num_peak = num_peaks(chunk,'2->1Bytes',1000000)
    #iman_up_num_peak = num_peaks(chunk,'1->2Bytes',50000)

    feat_val = np.hstack((
      dwl_spectral,
      dwl_agg,
      dwl_peak,
      upl_spectral,
      upl_agg,
      upl_peak,
      iman_dwn_time_peak,
      #iman_up_time_peak,
      iman_dwn_num_peak,
      #iman_up_num_peak
    ))
    
    vals.append(feat_val)
    
  return pd.DataFrame(columns=features, data=vals).fillna(0)

In [ ]:
def create_features_no_split(df, interval=60):

  features = [
    'peak_freq',
    'peak_prom',
    'max_psd',
    'bytes_avg',
    'bytes_std',
    'peak_avg',
    'peak_var',
    'peak_std',
  ]  

  vals = []

  df_chunks = chunk_data(df, interval)

  for chunk in df_chunks:

    preproc = convert_ms_df(chunk, True)

    ## spectral features
    spectral_feat = spectral_features(preproc, 'pkt_size')
    
    ## aggregate features
    aggr_feat = agg_feat(chunk, '2->1Bytes')
    
    ## peak features
    peak_feat = peak_time_diff(chunk, '2->1Bytes')
    
    feat_val = np.hstack((
      spectral_feat,
      aggr_feat,
      peak_feat
    ))
    
    vals.append(feat_val)
    
  return pd.DataFrame(columns=features, data=vals).fillna(0)

In [ ]:
low_feat_no_split = create_features_no_split(stdoan_low, 100)

## Dev Playground

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [10]:
%%time
low_feat = pd.concat([create_features(df, 100) for df in low_dfs])
med_feat = pd.concat([create_features(df, 100) for df in med_dfs])
high_feat = pd.concat([create_features(df, 100) for df in high_dfs])

Wall time: 45.5 s


In [11]:
low_feat['resolution'] = np.zeros(len(low_feat))
med_feat['resolution'] = np.zeros(len(med_feat)) + 1
high_feat['resolution'] = np.zeros(len(high_feat)) + 2

In [12]:
training_split = pd.concat([low_feat, med_feat, high_feat])

In [13]:
X, y = training_split.drop(columns=['resolution']), training_split['resolution']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 8)

In [14]:
clf_split = RandomForestClassifier(n_estimators = 5, max_depth = 2, criterion = 'entropy', random_state = 42)
clf_split.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [15]:
y_pred = clf_split.predict(X_test)

In [16]:
np.abs(training_split.corr()['resolution']).sort_values(ascending=False)

resolution            1.000000
dwl_max_psd           0.642118
dwl_peak_prom         0.640417
IMAN_dwn_num_peak     0.605758
dwl_bytes_avg         0.598011
dwl_bytes_std         0.582058
upl_peak_std          0.434709
upl_peak_var          0.395590
dwl_peak_std          0.391721
upl_bytes_avg         0.386697
dwl_peak_var          0.383599
dwl_peak_avg          0.371300
upl_peak_avg          0.356662
upl_peak_prom         0.315430
upl_max_psd           0.314997
upl_bytes_std         0.314364
IMAN_dwn_time_peak    0.235416
dwl_peak_freq         0.161576
upl_peak_freq         0.026870
Name: resolution, dtype: float64

In [17]:
(pd.crosstab(y_test, y_pred, rownames=['Actual Resolution'], colnames=['Predicted Resolution']))

Predicted Resolution,0.0,1.0,2.0
Actual Resolution,,,
0.0,14,1,0
1.0,3,8,0
2.0,2,0,12


In [18]:
f1_score(y_test, y_pred, average=None)

array([0.82352941, 0.8       , 0.92307692])

In [19]:
accuracy_score(y_test, y_pred)

0.85

In [21]:
features = [
  'dwl_peak_freq',
  'dwl_peak_prom',
  'dwl_max_psd',
  'dwl_bytes_avg',
  'dwl_bytes_std',
  'dwl_peak_avg',
  'dwl_peak_var',
  'dwl_peak_std',
  'upl_peak_freq',
  'upl_peak_prom',
  'upl_max_psd',
  'upl_bytes_avg',
  'upl_bytes_std',
  'upl_peak_avg',
  'upl_peak_var',
  'upl_peak_std',
  'IMAN_dwn_time_peak',#'IMAN_up_time_peak'
            'IMAN_dwn_num_peak']#,'IMAN_up_num_peak']
importances = clf_split.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print(features[i],': ',importances[i])
    

IMAN_dwn_time_peak :  0.22569869867018
IMAN_dwn_num_peak :  0.17516786569384635
dwl_bytes_std :  0.15965810104277414
upl_bytes_avg :  0.12010014165029204
upl_bytes_std :  0.11799720850901319
upl_max_psd :  0.0790332939856815
dwl_peak_prom :  0.0755371137334339
dwl_peak_std :  0.04680757671477871
dwl_peak_avg :  0.0
dwl_max_psd :  0.0
dwl_bytes_avg :  0.0
upl_peak_freq :  0.0
dwl_peak_var :  0.0
upl_peak_prom :  0.0
upl_peak_avg :  0.0
upl_peak_var :  0.0
upl_peak_std :  0.0
dwl_peak_freq :  0.0


In [20]:
import dill

In [21]:
dill.dump(clf_split, open("randomforest_chkpt2.obj", "wb"))

In [22]:
dill.load(open("randomforest_chkpt2.obj", "rb"))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

## no split (download focus)

In [ ]:
%%time
low_feat_no_split = create_features_no_split(stdoan_low, 100)
med_feat_no_split = create_features_no_split(stdoan_med, 100)
high_feat_no_split = create_features_no_split(stdoan_high, 100)

In [ ]:
low_feat_no_split['resolution'] = np.zeros(len(low_feat))
med_feat_no_split['resolution'] = np.zeros(len(med_feat)) + 1
high_feat_no_split['resolution'] = np.zeros(len(high_feat)) + 2

In [ ]:
training_no_split = pd.concat([low_feat_no_split, med_feat_no_split, high_feat_no_split])

In [ ]:
X, y = training_no_split.drop(columns=['resolution']), training_no_split['resolution']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 8)

In [ ]:
clf_no_split = RandomForestClassifier(n_estimators = 2, max_depth = 2, criterion = 'entropy', random_state = 42)
clf_no_split.fit(X_train, y_train)

In [ ]:
y_pred = clf_no_split.predict(X_test)

In [ ]:
np.abs(training_no_split.corr()['resolution']).sort_values(ascending=False)

In [ ]:
(pd.crosstab(y_test, y_pred, rownames=['Actual Resolution'], colnames=['Predicted Resolution']))

In [ ]:
f1_score(y_test, y_pred, average=None)